In [54]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/apple-stock-price-from-19802021/AAPL.csv


In [2]:
import pandas as pd

In [20]:
df=pd.read_csv('/kaggle/input/apple-stock-price-from-19802021/AAPL.csv')
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,1980-12-12,0.128348,0.128906,0.128348,0.128348,0.100178,469033600
1,1980-12-15,0.122210,0.122210,0.121652,0.121652,0.094952,175884800
2,1980-12-16,0.113281,0.113281,0.112723,0.112723,0.087983,105728000
3,1980-12-17,0.115513,0.116071,0.115513,0.115513,0.090160,86441600
4,1980-12-18,0.118862,0.119420,0.118862,0.118862,0.092774,73449600
...,...,...,...,...,...,...,...
10463,2022-06-13,132.869995,135.199997,131.440002,131.880005,131.880005,122207100
10464,2022-06-14,133.130005,133.889999,131.479996,132.759995,132.759995,84784300
10465,2022-06-15,134.289993,137.339996,132.160004,135.429993,135.429993,91533000
10466,2022-06-16,132.080002,132.389999,129.039993,130.059998,130.059998,108123900


In [22]:
df=df['Close']
df

0          0.128348
1          0.121652
2          0.112723
3          0.115513
4          0.118862
            ...    
10463    131.880005
10464    132.759995
10465    135.429993
10466    130.059998
10467    131.559998
Name: Close, Length: 10468, dtype: float64

In [36]:
import numpy as np

In [44]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
data=scaler.fit_transform(np.array(df).reshape(-1,1))
data

# apply scaling

array([[4.35483696e-04],
       [3.98684579e-04],
       [3.49613594e-04],
       ...,
       [7.44010911e-01],
       [7.14499102e-01],
       [7.22742631e-01]])

In [45]:
data_length=len(df)
test_data_length=int(data_length*0.2)
train_data_length=data_length-test_data_length
print("data_length: ",data_length,end='\n')
print("train_data_length: ",train_data_length,end='\n')
print("test_data_length: ",test_data_length,end='\n')

training_data=data[0:train_data_length]
testing_data=data[train_data_length:data_length]

#seperating training and testing data from data

data_length:  10468
train_data_length:  8375
test_data_length:  2093


In [49]:
training_data.shape

(8375, 1)

In [50]:
testing_data.shape

(2093, 1)

In [61]:
def timestep_data(data,time_step):
    x=[]
    y=[]
    for i in range(len(data)-time_step-1):
        x.append(data[i:(i+time_step),0])
        y.append(data[(i+time_step)+1,0])
    return np.array(x),np.array(y)

#function which seperate x,y from training and testing data

In [62]:
x_train,y_train=timestep_data(training_data,100)
x_test,y_test=timestep_data(testing_data,100)

#function calling

In [66]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(8274, 100)
(8274,)
(1992, 100)
(1992,)


In [68]:
import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM,Dense

In [69]:
model=Sequential()

model.add(LSTM(150,return_sequences=True,input_shape=(100,1)))
model.add(LSTM(100,return_sequences=True,input_shape=(100,1)))
model.add(LSTM(50,return_sequences=True,input_shape=(100,1)))
model.add(Dense(1,activation='linear'))

# Model architecture
# Here I used stacked LSTM
# Stacked LSTm means one after another LSTM layer

/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [70]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 100, 150)       │        91,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 100, 100)       │       100,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 100, 50)        │        30,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 100, 1)         │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 221,851 (866.61 KB)

 Trainable params: 221,851 (866.61 KB)

 Non-trainable params: 0 (0.00 B)

In [71]:
model.compile(loss='mean_squared_error',optimizer='adam',metrics=['accuracy'])

In [73]:
history=model.fit(x_train,y_train,validation_split=0.3,epochs=100)

Epoch 1/100
181/181 ━━━━━━━━━━━━━━━━━━━━ 46s 254ms/step - accuracy: 3.1290e-04 - loss: 3.4834e-06 - val_accuracy: 0.0000e+00 - val_loss: 0.0032
Epoch 2/100
181/181 ━━━━━━━━━━━━━━━━━━━━ 46s 253ms/step - accuracy: 9.7392e-05 - loss: 1.1228e-06 - val_accuracy: 0.0000e+00 - val_loss: 0.0032
Epoch 3/100
181/181 ━━━━━━━━━━━━━━━━━━━━ 45s 250ms/step - accuracy: 2.9627e-04 - loss: 1.2404e-06 - val_accuracy: 0.0000e+00 - val_loss: 0.0032
Epoch 4/100
181/181 ━━━━━━━━━━━━━━━━━━━━ 46s 255ms/step - accuracy: 2.5302e-05 - loss: 1.1521e-06 - val_accuracy: 0.0000e+00 - val_loss: 0.0033
Epoch 5/100
181/181 ━━━━━━━━━━━━━━━━━━━━ 45s 250ms/step - accuracy: 6.9860e-05 - loss: 1.1973e-06 - val_accuracy: 0.0000e+00 - val_loss: 0.0031
Epoch 6/100
181/181 ━━━━━━━━━━━━━━━━━━━━ 83s 254ms/step - accuracy: 1.7835e-05 - loss: 1.1643e-06 - val_accuracy: 0.0000e+00 - val_loss: 0.0032
Epoch 7/100
181/181 ━━━━━━━━━━━━━━━━━━━━ 46s 255ms/step - accuracy: 2.4215e-05 - loss: 1.0951e-06 - val_accuracy: 0.0000e+00 - val_loss: